### Day 16: Reindeer Maze

Link: https://adventofcode.com/2024/day/16#part2

To solve part two, we can modify the solution from part one as follows:

1. Include a track of unique visited positions in the reindeer data class. This updates the set of unique positions that lead to the goal with the lowest score.
2. Modify the set of visited places to a dictionary containing the score. We can allow the same position to be visited in the same direction multiple times now, since there can be multiple routes with the lowest possible score. However, we still want to avoid processing unnecessary paths. To do so, we skip states where the accumulated score is worse than the best score found for that state so far.
3. We only break the loop when the score being processed surpasses the lowest final score found, as that would mean future scores can only increase and there are no more optimal routes.

In [ ]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
import heapq
import itertools
import typing as t


T = t.TypeVar("T")


class HeapQueue(t.Generic[T]):
    def __init__(self, key: t.Optional[t.Callable[[T], float]] = None) -> None:
        self._heap: list[tuple[float, int, T]] = []
        self._key = key or (lambda x: x)
        self._counter = itertools.count()  # Unique counter to ensure stable ordering

    def add(self, item: T) -> None:
        count = next(self._counter)
        heapq.heappush(self._heap, (self._key(item), count, item))

    def remove(self) -> T:
        if self.is_empty():
            raise IndexError("Empty heap")

        return heapq.heappop(self._heap)[-1]

    def peek(self) -> T:
        if self.is_empty():
            raise IndexError("Empty heap")

        return self._heap[0][-1]

    def is_empty(self) -> bool:
        return len(self._heap) == 0

    def __len__(self) -> int:
        return len(self._heap)

In [ ]:
from dataclasses import dataclass


grid: list[list[str]] = []


for line in lines:
    grid_row = list(line.strip())
    grid.append(grid_row)


def find_start_position() -> tuple[int, int]:
    for row in range(len(grid)):
        for column, cell in enumerate(grid[row]):
            if cell == "S":
                return row, column

    raise Exception("Start position not found")


@dataclass
class Reindeer:
    row: int
    column: int
    direction: int
    score: int
    track: set[tuple[int, int]]


row, column = find_start_position()
actions = [
    (0, 1), # Right
    (1, 0), # Down
    (0, -1), # Left
    (-1, 0), # Up
]
queue: HeapQueue[Reindeer] = HeapQueue(key=lambda reindeer: reindeer.score)
queue.add(Reindeer(row=row, column=column, direction=0, score=0, track={(row, column)}))


def turn_right(direction: int) -> int:
    return (direction + 1) % len(actions)


def turn_left(direction: int) -> int:
    return (direction - 1) % len(actions)


visited: dict[tuple[int, int, int], int] = {}
min_score = float("inf")
optimal_track: set[tuple[int, int]] = set()


while not queue.is_empty():
    reindeer = queue.remove()

    if reindeer.score > visited.get((reindeer.row, reindeer.column, reindeer.direction), float("inf")):
        continue

    visited[(reindeer.row, reindeer.column, reindeer.direction)] = reindeer.score

    if reindeer.score > min_score:
        break

    if grid[reindeer.row][reindeer.column] == "E":
        min_score = reindeer.score
        optimal_track |= reindeer.track
        continue

    new_row, new_column = reindeer.row + actions[reindeer.direction][0], reindeer.column + actions[reindeer.direction][1]

    if grid[new_row][new_column] != "#":
        queue.add(Reindeer(
            row=new_row,
            column=new_column,
            direction=reindeer.direction,
            score=reindeer.score + 1,
            track=reindeer.track | {(new_row, new_column)},
        ))

    queue.add(Reindeer(
        row=reindeer.row,
        column=reindeer.column,
        direction=turn_right(reindeer.direction),
        score=reindeer.score + 1_000,
        track=reindeer.track,
    ))
    queue.add(Reindeer(
        row=reindeer.row,
        column=reindeer.column,
        direction=turn_left(reindeer.direction),
        score=reindeer.score + 1_000,
        track=reindeer.track,
    ))


print(len(optimal_track))